In [ ]:
# import packages used in this notebook
import os
import json

In [ ]:
tweets = [] # list of tweets

In [ ]:
def getbz2Tweets(files, subject):
    import bz2
    import re
    subj = subject.lower()
    with bz2.open(filbz, "rt", encoding="utf-8") as f:
    # error encountered when loading the entire file as json object
        for line in f:
            try:
                data = json.loads(line)
                # filter the data for my subject
                fltrData = re.search(r'\b{}\b'.format(subj), line.lower())

                if fltrData:
                    try:
                        tweet = {'id':[data["id"]],
                              'text': data["text"]}
                
                        # user = data["user"]
                        # tweet['userName'] = user["name"]
                        # tweet['userLocation'] = user["location"]
                        # tweet['userFollowers'] = user["followers_count"]
                        # if 'retweeted_status' in data:
                        #     retweet = data["retweeted_status"]
                        #     tweet['retweetUserName'] = retweet["user"]
                        #     tweet['retweetCount'] = retweet["retweet_count"]
                        tweets.append(tweet)
                    except KeyError as e:
                        #print("KeyError encountered", {e}) # error is encountered mainly due to deleted tweets 
                        #print(line)
                        continue
            except json.decoder.JSONDecodeError as e:
                print("Error encountered at line: ", {e})
                continue
    return tweets
   

In [ ]:
def getgzTweets(files, subject):
    import gzip
    import re
    subj = subject.lower()
    with gzip.open(files, "rt", encoding="utf-8") as f:
    # error encountered when loading the entire file as json object
        for line in f:
            try:
                data = json.loads(line)
                # filter the data for my subject
                fltrData = re.search(r'\b{}\b'.format(subj), line.lower())

                if fltrData:
                    print(line)
                    try:
                        tweet = {'id':[data["id"]],
                              'text': data["text"]}
                
                        # user = data["user"]
                        # tweet['userName'] = user["name"]
                        # tweet['userLocation'] = user["location"]
                        # tweet['userFollowers'] = user["followers_count"]
                        # if 'retweeted_status' in data:
                        #     retweet = data["retweeted_status"]
                        #     tweet['retweetUserName'] = retweet["user"]
                        #     tweet['retweetCount'] = retweet["retweet_count"]
                        tweets.append(tweet)
                    except KeyError as e:
                        #print("KeyError encountered", {e}) # error is encountered mainly due to deleted tweets 
                        #print(line)
                        continue
            except json.decoder.JSONDecodeError as e:
                print("Error encountered at line: ", {e})
                continue
    return tweets
   

In [ ]:
tweetFiles = os.listdir('./SampleTweets/archive')
for filbz in tweetFiles:
    filbz = './SampleTweets/archive/' + filbz
    if filbz.endswith('.bz2'):
        tweets.append(getbz2Tweets(filbz, 'euro'))
    elif filbz.endswith('.gz'):
        tweets.append(getgzTweets(filbz, 'euro'))

In [ ]:
display(tweets[0]['id'])

In [ ]:
idList = [tweet['id'] for tweet in tweets]
if [1410485572347387910] in idList:
    print("True")

In [ ]:
display(tweets)

In [ ]:
type(tweets[0])